In [1]:
#Mounting google and authenticating
from google.colab import drive
drive.mount('/content/gdrive')

!ln -s /content/gdrive/My\ Drive/ /mydrive
#!ls /mydrive

Mounted at /content/gdrive


In [2]:
!cp /mydrive/music\ project/X_spec.pickle ./
!cp /mydrive/music\ project/y_spec.pickle ./

In [3]:
#LOADING DATA
import pickle 
pickle_off = open("X_spec.pickle", 'rb')
X_spec = pickle.load(pickle_off)

pickle_off = open("y_spec.pickle", 'rb')
y_spec = pickle.load(pickle_off)

In [5]:
type(X_spec)

numpy.ndarray

In [7]:
import tensorflow as tf

dataset = tf.data.Dataset.from_tensor_slices([X_spec, y_spec])

In [21]:
import numpy as np
dataset = np.hstack((X_spec,y_spec))

In [22]:
dataset.shape

(1431, 600, 128)

In [23]:
new = dataset.reshape(1431,2,300,128)

In [ ]:
#for train
import tensorflow as tf

dataset = tf.data.Dataset.from_tensor_slices(X_spec)

def normalize(input_image, input_mask):
    input_image = tf.cast(input_image, tf.float32) /2
    #input_image = tf.image.rgb_to_grayscale(input_image)
    input_mask = tf.cast(input_mask, tf.float32) 
    #input_mask = tf.image.rgb_to_grayscale(input_mask)
    return input_image, input_mask

def load_image_train(datapoint):
    #input_image = tf.image.resize(datapoint[0], (128, 128))
    #input_mask = tf.image.resize(datapoint[1], (128, 128))

    input_image, input_mask = normalize(datapoint[0], datapoint[1])

    return input_image, input_mask

train = dataset.map(load_image_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)


In [ ]:
#for test in tensorflow - getting ready for fitting

testd = tf.data.Dataset.from_tensor_slices(new_t)

def normalize_test(input_image, input_mask):
    input_image = tf.cast(input_image, tf.float32) /2
    #input_image = tf.image.rgb_to_grayscale(input_image) 
    input_mask = tf.cast(input_mask, tf.float32) 
    #input_mask = tf.image.rgb_to_grayscale(input_mask)
    return input_image, input_mask

def load_image_test(datapoint):
    #input_image = tf.image.resize(datapoint[0], (128, 128))
    #input_mask = tf.image.resize(datapoint[1], (128, 128))

    input_image, input_mask = normalize_test(datapoint[0], datapoint[1])

    return input_image, input_mask

test = testd.map(load_image_test)